# Chapter 11. 인터페이스: 프로토콜에서 ABC까지

## 11.1 파이썬 문화에서의 인터페이스와 프로토콜

In [1]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [2]:
class Vectro2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y

In [4]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()
f[1]

10

In [6]:
for i in f: print(i)

0
10
20


In [7]:
20 in f

True

In [9]:
15 in f

False

In [32]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
        
    def __getitem__(self, position):
        return self._cards[position]

## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭

In [21]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[4, 6, 5, 0, 8, 3, 2, 7, 9, 1]

In [33]:
deck = FrenchDeck()

shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [35]:
def set_card(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='3', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='Q', suit='hearts'),
 Card(rank='2', suit='hearts'),
 Card(rank='4', suit='hearts')]

## 11.4 알렉스 마르텔리의 물새

In [37]:
from collections import abc

class Struggle:
    def __len__(self):
        return 23
    
isinstance(Struggle(), abc.Sized)

True

## 11.5 ABC 상속하기

In [46]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, card):
        self._cards[position] = card
        
    def __delitem__(self, position):
        del self._cards[position]
        
    def insert(self, position, card):
        self._cards.insert(position, card)
        
FrenchDeck2()

In [47]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, card):
        self._cards[position] = card
        
#     def __delitem__(self, position):
#         del self._cards[position]
        
#     def insert(self, position, card):
#         self._cards.insert(position, card)
        
FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract methods __delitem__, insert

## 11.6.2 표준 라이브러리의 ABC

In [6]:
import collections.abc
print(dir(collections.abc))

['AsyncGenerator', 'AsyncIterable', 'AsyncIterator', 'Awaitable', 'ByteString', 'Callable', 'Collection', 'Container', 'Coroutine', 'Generator', 'Hashable', 'ItemsView', 'Iterable', 'Iterator', 'KeysView', 'Mapping', 'MappingView', 'MutableMapping', 'MutableSequence', 'MutableSet', 'Reversible', 'Sequence', 'Set', 'Sized', 'ValuesView', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [12]:
k = {'a':1}.keys()
k.__class__

dict_keys

## 11.7 ABC의 정의와 사용

In [38]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        """ add iterable instance"""
    
    @abc.abstractmethod
    def pick(self):
        """ random selection
        raise LookupError if this is empty
        """ 
        
    def loaded(self):
        return bool(self.inspect())
    
    def inspect(self):
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
            
        self.load(items)
        return tuple(sorted(items))
    

In [16]:
class Fake(Tombola):
    def pick(self):
        return 13
    
Fake

__main__.Fake

In [17]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [19]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, _):
        pass

In [39]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()
        
bingo_cage = BingoCage([1,2,3])
bingo_cage.loaded()


True

In [42]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls[position]
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))
    
lottery_blower = LotteryBlower([1, 2, 3])
lottery_blower.inspect()

(1, 2, 3)

In [46]:
@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self[position]
        else:
            raise LookupError('pick from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
    
tombolist = TomboList([1,2,3])
tombolist.inspect()

(1, 2, 3)

In [48]:
issubclass(TomboList, Tombola)

True

In [49]:
isinstance(TomboList(), Tombola)

True

In [57]:
TomboList.__mro__

(__main__.TomboList, list, object)

## 11.8 Tombola 서브클래스 테스트 방법

In [58]:
Tombola.__subclasses__()

[__main__.BingoCage, __main__.LotteryBlower, __main__.LotteryBlower]

In [67]:
Tombola._dump_registry()

Class: __main__.Tombola
Inv. counter: 48
_abc_registry: {<weakref at 0x104bf4868; to 'type' at 0x7fcb91524a98 (TomboList)>}
_abc_cache: set()
_abc_negative_cache: set()
_abc_negative_cache_version: 48


## 11.10 오리처럼 행돌할 수 있는 거위

In [68]:
class Struggle:
    def __len__(self):
        return 23
    
from collections import abc

isinstance(Struggle(), abc.Sized)

True

In [69]:
issubclass(Struggle, abc.Sized)

True

In [77]:
import abc

class Sized(abc.ABC):
    __slots__ = ()
    
    @abc.abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented